# Курсовой проект по курсу "Python для DataScience часть 2"

## Импорт библиотек

In [ ]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

In [ ]:
test_path = 'course_project_test.csv'
train_path = 'course_project_train.csv'

## Построение модели классификации

### 1. Обзор обучающего датасета

In [ ]:
df_train = pd.read_csv(train_path)
df_train.head()

In [ ]:
df_train.shape

Видим, что имеется 7500 наблюдений и 17 признаков, из них один признак - Credit Default - целевой.

In [ ]:
df_train.iloc[0]

Описание датасета:

    Home Ownership - домовладение,
    Annual Income - годовой доход,
    Years in current job - количество лет на текущем месте работы,
    Tax Liens - налоговые обременения,
    Number of Open Accounts - количество открытых счетов,
    Years of Credit History - количество лет кредитной истории,
    Maximum Open Credit - наибольший открытый кредит,
    Number of Credit Problems - количество проблем с кредитом,
    Months since last delinquent - количество месяцев с последней просрочки платежа,
    Bankruptcies - банкротства,
    Purpose - цель кредита,
    Term - срок кредита,
    Current Loan Amount - текущая сумма кредита,
    Current Credit Balance - текущий кредитный баланс,
    Monthly Debt - ежемесячный долг,
    Credit Default - факт невыполнения кредитных обязательств (0 - погашен вовремя, 1 - просрочка)

In [ ]:
df_train.info()

Видим (там, где менее 7500), что есть пропуски. Это касается 'Annual Income', т.е. не у всех известен годовой доход. Что касается поля 'Month since last delinquent', то пропуски (NaN) можно заменить большим числом (фактически не было правонарушений). Таким числом может быть практически бесконечный срок кредита - 50 лет (600 месяцев).

In [ ]:
df_train.describe()

В целом, каких-то заоблачных значений нет, за исключением, возможно, суммы наибольшего открытого кредита кредита около 1300 млн. руб или текущей суммы кредита около 100 млн. руб. Из таблицы видно, что среднее значение 'Current Default' равно 0.28, что говорит о несбалансированности выборки (в данной выборке "дефолтников" около 28%).

### 2. Обработка выбросов

In [ ]:
col_name = 'Current Loan Amount'
ax = sns.boxplot(y = df_train[col_name])

Путем перебора выясняем, что подозрительным является значение "все девятки"

In [ ]:
val = 99999999
sum(df_train[col_name] == val)

Посмотрим на статистику данных в этом подозрительном разрезе

In [ ]:
df_train[df_train[col_name] == val][set(df_train.columns) - {col_name}].describe()

Видим, что это "недефолтники" (max и min 'Credit Default' равны 0), а так как недефолтников в выборке больше, то удаление части их не так страшно (тех, у которых подозрительное значение 'Current Loan Amount' 99999999).

Проверим признак 'Maximum Open Credit'

In [ ]:
col_name = 'Maximum Open Credit'
ax = sns.boxplot(y = df_train[col_name])

In [ ]:
val = 1304726170
sum(df_train[col_name] == val)

In [ ]:
df_train[df_train[col_name] == val][set(df_train.columns) - {col_name}]

Ничего особенного, но так как такое значение одно, то принимаем это за выброс и удаляем.

In [ ]:
df_train = df_train.drop(df_train[df_train[col_name] == val].index)

In [ ]:
val = 380052288
sum(df_train[col_name] == val)

In [ ]:
df_train[df_train[col_name] == val][set(df_train.columns) - {col_name}]

Ничего особенного. Опять удаляем выброс.

In [ ]:
df_train = df_train.drop(df_train[df_train[col_name] == val].index)

In [ ]:
val = 265512874
sum(df_train[col_name] == val)

In [ ]:
df_train[df_train[col_name] == val][set(df_train.columns) - {col_name}]

Ничего особенного.
Итого: удаляем наблюдения, у которых признак 'Current Loan Amount' равен 99999999.

In [ ]:
df_train = df_train.drop(df_train[df_train[col_name] == val].index)

In [ ]:
col_name = 'Current Loan Amount'
val = 99999999
df_train = df_train.drop(df_train[df_train[col_name] == val].index)

In [ ]:
df_train.describe()

Видим, что баланс выборки несколько улучшился (32% дефолтников), но ценой удаления 870 наблюдений (12%).

### 3. Обработка пропусков

Как уже говорилось ранее, заменим NaN в признаке 'Months since last delinquent' на 600 месяцев.

In [ ]:
col_name = 'Months since last delinquent'
val = 600
df_train[col_name].fillna(val, inplace = True)

In [ ]:
df_train.describe()

NaN в признаке 'Annual Income' заменим условной медианой (условной, т.е. зависящей от 'Credit Default').

In [ ]:
print(df_train[df_train['Credit Default'] == 0]['Annual Income'].median())
print(df_train[df_train['Credit Default'] == 1]['Annual Income'].median())

In [ ]:
col_name = 'Annual Income'
val_0 = df_train[df_train['Credit Default'] == 0][col_name].median()
val_1 = df_train[df_train['Credit Default'] == 0][col_name].median()
df_train.loc[df_train['Credit Default'] == 0, col_name] = df_train.loc[df_train['Credit Default'] == 0, col_name].fillna(val_0)
df_train.loc[df_train['Credit Default'] == 1, col_name] = df_train.loc[df_train['Credit Default'] == 1, col_name].fillna(val_1)

In [ ]:
df_train.describe()

Видим, что не обработан признак 'Bankruptcies'.

In [ ]:
col_name = 'Bankruptcies'
df_train[pd.isnull(df_train[col_name])][set(df_train.columns) - set(col_name)].head()

Удаляем эти наблюдения.

In [ ]:
df_train.dropna(subset = [col_name], inplace = True)

In [ ]:
df_train.describe()

Остался признак 'Credit Score'. Посмотрим на его значимость (корреляцию с другими, в т.ч с целевой переменной)

In [ ]:
corr_with_target = df_train.corr().iloc[:-1, -1].sort_values(ascending=False)

plt.figure(figsize=(12, 4))

sns.barplot(x=corr_with_target.values, y=corr_with_target.index)

plt.title('Correlation with target variable')
plt.show()

In [ ]:
col_name = 'Credit Default'
with_name = 'Credit Score'
df_tmp = df_train.copy()
df_tmp.dropna(subset = [with_name], inplace = True)
limit_bal_with_target_s = df_tmp[[with_name, col_name]]

sns.jointplot(x = col_name, y = with_name, data = limit_bal_with_target_s)
plt.show()

Видим, что 'Credit Score' в данном случае разделен на две области; но, к счастью, оказалось достаточным большие значения разделить на 10, получив восстановленные значения.

In [ ]:
col_name = 'Credit Score'
threshold = 5000
df_train.loc[df_train[col_name] > threshold, col_name] = df_train[df_train[col_name] > threshold][col_name] * 0.1

In [ ]:
col_name = 'Credit Default'
with_name = 'Credit Score'
df_tmp = df_train.copy()
df_tmp.dropna(subset = [with_name], inplace = True)
limit_bal_with_target_s = df_tmp[[with_name, col_name]]

sns.jointplot(x = col_name, y = with_name, data = limit_bal_with_target_s)
plt.show()

In [ ]:
col_name = 'Credit Score'
plt.figure(figsize=(12, 4))
sns.kdeplot(df_tmp.loc[(df_tmp['Credit Default'] == 0), col_name], label = 'No Default')
sns.kdeplot(df_tmp.loc[(df_tmp['Credit Default'] == 1), col_name], label = 'Default')

plt.title('Credit Score w(cs)')
plt.show()

In [ ]:
col_name = 'Credit Score'
val_0 = df_train[df_train['Credit Default'] == 0][col_name].median()
val_1 = df_train[df_train['Credit Default'] == 0][col_name].median()
df_train.loc[df_train['Credit Default'] == 0, col_name] = df_train.loc[df_train['Credit Default'] == 0, col_name].fillna(val_0)
df_train.loc[df_train['Credit Default'] == 1, col_name] = df_train.loc[df_train['Credit Default'] == 1, col_name].fillna(val_1)

In [ ]:
print(val_0)
print(val_1)

In [ ]:
df_train.describe()

### 4. Анализ данных

Выделим категориальные переменные.

In [ ]:
df_train.head()

In [ ]:
df_train.info()

In [ ]:
cat_col_names = list(df_train.select_dtypes(include = ['object']).columns)
print('Категориальные признаки')
print(cat_col_names)
tgt_col_names = ['Credit Default']
val_col_names = list(set(df_train.columns) - set(cat_col_names) - set(tgt_col_names))
print('Числовые признаки')
print(val_col_names)

Посмотрим какие значения могут принимать категориальные переменные

In [ ]:
for name in cat_col_names:
    print(name)
    print('*************')
    print(set(df_train[name]))

Видим значение NaN в признаке 'Years in current job'; видимо, это означает то, что клиент на данный момент не работает. Категорий много - 33.

Проанализируем корреляцию признаков с целевой переменной в разрезе категориальных переменных, определенных выше.

In [ ]:
# Функция для построения корреляций признаков с целевой переменной в разрезах значений категориальной переменной col_name
def graph_corr(col_name, data, sz):
    cols = list(set(df_train[col_name]))
    cols = [str(c) for c in cols]
    for p in cols:
        eq = df_train[col_name].astype('str') == p
        count = sum(eq)
        corr_with_target = df_train[eq].corr().iloc[:-1, -1].sort_values(ascending = False)
        plt.figure(figsize = sz)
        sns.barplot(x = corr_with_target.values, y = corr_with_target.index)
        plt.title('Correlation with target variable. ' + col_name + ' = ' + str(p) + '. Count = ' + str(count))
        plt.xlim([-1, 1])
        plt.show()

- Домовладение

In [ ]:
col_name = 'Home Ownership'
graph_corr(col_name, df_train, (12,4))

- Количество лет на текущей работе

In [ ]:
col_name = 'Years in current job'
graph_corr(col_name, df_train, (12,4))

- Цель

In [ ]:
col_name = 'Purpose'
graph_corr(col_name, df_train, (12,4))

- Срочность

In [ ]:
col_name = 'Term'
graph_corr(col_name, df_train, (12,4))

В целом вывод таков, что разбиение на категории играет большую роль, и этим пренебрегать не следует, даже если наблюдений мало. 

Матрица корреляций:

In [ ]:
plt.figure(figsize = (24, 24))

bs_fn = df_train.columns.drop('Credit Default').tolist()

sns.set(font_scale = 1.4)
sns.heatmap(df_train[bs_fn].corr().round(3), annot = True, linewidths = 0.5, cmap = 'GnBu')

plt.title('Correlation matrix')
plt.show()

In [ ]:
plt.figure(figsize = (10, 8))

col_name = 'Credit Default'
sns.countplot(x = 'Tax Liens', hue = col_name, data = df_train)
plt.title('Tax Liens grouped by target variable')
plt.legend(title = 'Target', loc = 'upper right')

plt.show()

Видим, что в основном признак 'Tax Liens' равен 0 или 1.

In [ ]:
col_name = 'Credit Default'
plt.figure(figsize = (12, 4))
sns.kdeplot(df_train.loc[(df_train['Tax Liens'] == 0), col_name], label = 'TL 0')
sns.kdeplot(df_train.loc[(df_train['Tax Liens'] == 1), col_name], label = 'TL 1')

plt.title('Target')
plt.show()

Видим, что пропорции Default/No default примерно одинаковы как при TL = 0, так и при TL = 1, поэтому признак 'Tax Liens' исключаем.

### 5. Отбор признаков

Еще раз взглянем на корреляцию признаков с целевой переменной 'Credit Default'

In [ ]:
corr_with_target = df_train.corr().iloc[:-1, -1].sort_values(ascending=False)

plt.figure(figsize=(12, 4))

sns.barplot(x = corr_with_target.values, y = corr_with_target.index)

plt.title('Correlation with target variable')
plt.show()

Видим, что в общем и целом 'Credit Score' антикоррелирован с целевой переменной, что логично (чем выше 'Credit Score', тем вероятнее выдача кредита, т.е. 'Credit Default' = 0). Также и годовой доход, 'Annual Income'. Признак 'Bankruptcies' не коррелирует с целевой переменной, но это еще не означает независимости.

Посмотрим на разные сочетания пар признаков.

In [ ]:
col_1 = 'Bankruptcies'
col_2 = 'Credit Default'
show_joint = True

Совместная плотность w(x,y)

In [ ]:
tmp = df_train[[col_1, col_2]]

if show_joint:
    sns.jointplot(x = col_1, y = col_2, data = tmp)
    plt.show()

Безусловная плотность w(x)

In [ ]:
tmp = df_train[col_1]
sns.distplot(tmp, norm_hist = 'True')

plt.show()

Безусловная плотность w(y)

In [ ]:
tmp = df_train[col_2]
sns.distplot(tmp, norm_hist = 'True')

plt.show()

Так как "на глазок" w(x,y) = w(x) w(y), то можно сделать вывод о независимости 'Credit Default' от 'Bankruptcies' и удалить 'Bankruptcies', тем более он коррелирует (r = 0.73) с 'Number of Credit Problems'. Дальше пойдем более простым путем: будем оценивать условные среднее значение и среднеквадратическое отклонение и их безусловные значения (т.е. оценки плотностей заменим на оценки двух моментов).

In [ ]:
col_1 = 'Monthly Debt'
col_2 = 'Credit Default'

print(df_train[df_train[col_2] == 0][col_1].mean())
print(df_train[df_train[col_2] == 1][col_1].mean())
print(df_train[col_1].mean())

print(df_train[df_train[col_2] == 0][col_1].std())
print(df_train[df_train[col_2] == 1][col_1].std())
print(df_train[col_1].std())

Видим, что условные от безусловного не особенно отличаются (критериями согласия пренебрегаем), значит можно пренебречь признаком 'Monthly Debt'.

In [ ]:
col_1 = 'Months since last delinquent'
col_2 = 'Credit Default'

print(df_train[df_train[col_2] == 0][col_1].mean())
print(df_train[df_train[col_2] == 1][col_1].mean())
print(df_train[col_1].mean())

print(df_train[df_train[col_2] == 0][col_1].std())
print(df_train[df_train[col_2] == 1][col_1].std())
print(df_train[col_1].std())

Аналогично пренебрегаем признаком 'Months since last delinquent'.

In [ ]:
col_1 = 'Current Credit Balance'
col_2 = 'Credit Default'

print(df_train[df_train[col_2] == 0][col_1].mean())
print(df_train[df_train[col_2] == 1][col_1].mean())
print(df_train[col_1].mean())

print(df_train[df_train[col_2] == 0][col_1].std())
print(df_train[df_train[col_2] == 1][col_1].std())
print(df_train[col_1].std())

А вот признаком 'Current Credit Balance' пренебрегать не следует.

In [ ]:
cols_for_drop = ['Bankruptcies', 'Monthly Debt', 'Months since last delinquent', 'Tax Liens']
try:
    df_train.drop(columns = cols_for_drop, inplace = True)
except:
    print('Columns were deleted')

df_train.head()

In [ ]:
df_train.describe()

In [ ]:
df_train.info()

Обзор object-признаков

In [ ]:
for cat_colname in df_train.select_dtypes(include = 'object').columns:
    print(str(cat_colname) + '\n\n' + str(df_train[cat_colname].value_counts()) + '\n' + '*' * 100 + '\n')

То есть чаще всего кредит берут краткосрочный, для того чтобы создать единый кредитный счет (debt consolidation) с пониженной ставкой, при этом работают на одной работе более 10 лет и имеют ипотеку.

Проверка на пропуски

In [ ]:
len(df_train) - df_train.count()

Как мы уже говорили, NaN в поле 'Years in current job' пропуском не считается. Однако, заменим NaN на 'None' (нет текущей работы).

In [ ]:
col_name = 'Years in current job'
df_train[col_name].fillna('None', inplace = True)

In [ ]:
len(df_train) - df_train.count()

4. Анализ данных\n",
    "5. Отбор признаков\n",
    "6. Балансировка классов\n",
    "7. Подбор моделей, получение бейзлана\n",
    "8. Выбор наилучшей модели, настройка гиперпараметров\n",
    "9. Проверка качества, борьба с переобучением\n",
    "10. Интерпретация результатов\n",
    "\n",
    "**Прогнозирование на тестовом датасете**\n",
    "1. Выполнить для тестового датасета те же этапы обработки и постронияния признаков\n",
    "2. Спрогнозировать целевую переменную, используя модель, построенную на обучающем датасете\n",
    "3. Прогнозы должны быть для всех примеров из тестового датасета (для всех строк)\n",
    "4. Соблюдать исходный порядок примеров из тестового датасета"

In [ ]:
df_train.info()

Выделение dummy-переменных

In [ ]:
for c in df_train.select_dtypes(include = 'object').columns:
    df_train = pd.concat([df_train, pd.get_dummies(df_train[c], prefix = c)], axis=1)

In [ ]:
df_train.info()

In [ ]:
df_train.head()

Сохранение обучающего датасета

In [ ]:
df_train.to_csv('prep_train.csv', index = False, encoding = 'utf-8')

### 6. Балансировка классов

In [ ]:
def balance_df_by_target(df, target_name):
    target_counts = df[target_name].value_counts()

    major_class_name = target_counts.idxmax()
    minor_class_name = target_counts.idxmin()

    disbalance_coeff = int(target_counts[major_class_name] / target_counts[minor_class_name]) - 1

    for i in range(disbalance_coeff):
        sample = df[df[target_name] == minor_class_name].sample(target_counts[minor_class_name])
        df = df.append(sample, ignore_index = True)

    return df.sample(frac = 1) 

In [ ]:
col_name = 'Credit Default'
df_balanced = balance_df_by_target(df_train, col_name)

После балансировки:

In [ ]:
df_balanced[col_name].value_counts()

До балансировки:

In [ ]:
df_train[col_name].value_counts()

### 7. Подбор моделей

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, ShuffleSplit, cross_val_score, learning_curve
from sklearn.model_selection import KFold, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb, lightgbm as lgbm, catboost as catb

In [ ]:
df_base = pd.read_csv(train_path)

Нормализация данных

In [ ]:
NUMERIC_FEATURE_NAMES = df_train.select_dtypes(include = 'float64').columns.to_list()
TARGET_NAME = 'Credit Default'

scaler = StandardScaler()

df_norm = df_balanced.copy()
df_norm[NUMERIC_FEATURE_NAMES] = scaler.fit_transform(df_norm[NUMERIC_FEATURE_NAMES])

df = df_norm.copy()

In [ ]:
def get_classification_report(y_train_true, y_train_pred, y_test_true, y_test_pred):
    print('TRAIN\n\n' + classification_report(y_train_true, y_train_pred))
    print('TEST\n\n' + classification_report(y_test_true, y_test_pred))
    print('CONFUSION MATRIX\n')
    print(pd.crosstab(y_test_true, y_test_pred))

In [ ]:
try:
    BASE_FEATURE_NAMES = list(set(df_base.columns.drop(TARGET_NAME).tolist()) - set(cols_for_drop))
except:
    print('Columns were droped')
try:
    NEW_FEATURE_NAMES = df.columns.drop([TARGET_NAME] + BASE_FEATURE_NAMES).tolist()
except:
    print('Columns were droped')
FEATURE_NAMES_SELECTED = NUMERIC_FEATURE_NAMES + NEW_FEATURE_NAMES

In [ ]:
NUMERIC_FEATURE_NAMES

In [ ]:
NEW_FEATURE_NAMES

In [ ]:
X = df[FEATURE_NAMES_SELECTED]
Y = df[TARGET_NAME]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, shuffle = True, test_size = 0.25, random_state = 231)

Логистическая регрессия

In [ ]:
model_lr = LogisticRegression()
model_lr.fit(X_train, Y_train)

Y_train_pred = model_lr.predict(X_train)
Y_test_pred = model_lr.predict(X_test)

get_classification_report(Y_train, Y_train_pred, Y_test, Y_test_pred)

## Прогнозирование на тестовом датасете

In [ ]:
pd.get_dummies(df_train['Home Ownership'], prefix = 'Home Ownership').info()